# 0.0. Imports

## 0.1. Imports

In [9]:
import requests
import pandas as pd
import re

# 0.2. Helper Functions

In [29]:
def create_mape_rmse_columns(df):
    # Only runned's
    df = df[df['runned'] == 1]
    # Transform columns in a number
    df['mean_MAPE'] = df['MAPE'].apply(lambda x: float(re.match('(.*) \+/- (.*)', x).group(1)))
    df['std_MAPE'] = df['MAPE'].apply(lambda x: float(re.match('(.*) \+/- (.*)', x).group(2)))

    df['mean_RMSE'] = df['RMSE'].apply(lambda x: float(re.match('(.*) \+/- (.*)', x).group(1)))
    df['std_RMSE'] = df['RMSE'].apply(lambda x: float(re.match('(.*) \+/- (.*)', x).group(2)))
    
    return df

def display_errors_by_parameters(df, list_of_parameters):
    # printing 
    for column in list_of_parameters:
        display(df[[column, 'mean_MAPE', 'std_MAPE', 'mean_RMSE', 'std_RMSE']].groupby(column).mean())
    
    return None

#  1.0. GETTING DATA

In [11]:
response = requests.get('https://xgboostfinetuning.miguelzaq.repl.co/data')
file_json = response.json()
data = pd.DataFrame(file_json)
# save for future use
data.to_csv('../data_params/outcome_random.csv')

# if API -- offilne 
#data = pd.read_csv('../data_params/outcome_first_random.csv')

# 2.0. PARAMS

In [12]:
param = {
    'n_estimatores': [800, 1300, 2100],
    'learning_rate': [0.03, 0.05, 0.08], 
    'max_depth': [3, 6, 9],
    'subsample': [0.5, 0.75, 1],  
    'colsample_bytee': [0.5, 0.75, 1],
    'colsample_bynode': [0.5, 0.75, 1],
    'min_child_weight': [1, 3, 5]
}

## 2.1. FIRST ROUND

In [18]:
df00 = data.copy()

In [19]:
# only the first run
df00 = df00[df00['id'] < 10000]

df00 = create_mape_rmse_columns(df00)

# filter best 50 of 173
df00 = df00.loc[df00['mean_MAPE'].sort_values().index]
df00 = df00.iloc[:50]

list_columns = [
    'n_estimatores', 'learning_rate', 'max_depth', 'subsample',
    'colsample_bytee', 'colsample_bynode', 'min_child_weight'
]

display_errors_by_parameters(df00, list_columns)

C:\Users\Miguel\AppData\Local\Temp/ipykernel_1560/3186266846.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean_MAPE'] = df['MAPE'].apply(lambda x: float(re.match('(.*) \+/- (.*)', x).group(1)))
C:\Users\Miguel\AppData\Local\Temp/ipykernel_1560/3186266846.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std_MAPE'] = df['MAPE'].apply(lambda x: float(re.match('(.*) \+/- (.*)', x).group(2)))
C:\Users\Miguel\AppData\Local\Temp/ipykernel_1560/3186266846.py:8: SettingWithCopyWarning: 
A value is 

,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
n_estimatores,,,,
800,0.194444,0.012222,2086.146667,320.805556
1300,0.202273,0.016364,2151.906364,325.942273
2100,0.195263,0.017368,2074.985263,319.084211


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
learning_rate,,,,
0.05,0.200000,0.015625,2212.778125,356.705000
0.08,0.197353,0.016176,2062.868529,306.273529


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
max_depth,,,,
6,0.219474,0.016842,2281.924211,307.075789
9,0.185161,0.015484,2005.981290,331.810968


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
subsample,,,,
0.50,0.197059,0.015882,2101.648824,329.453529
0.75,0.195000,0.015000,2076.769375,328.710625
1.00,0.202353,0.017059,2152.096471,309.441176


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
colsample_bytee,,,,
0.50,0.194375,0.016875,2051.575000,319.255625
0.75,0.197222,0.016667,2114.421111,326.438333
1.00,0.203125,0.014375,2166.075000,321.037500


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
colsample_bynode,,,,
0.50,0.190000,0.015000,2013.012778,312.238889
0.75,0.200556,0.014444,2142.740556,319.936667
1.00,0.205714,0.019286,2195.601429,338.672857


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
min_child_weight,,,,
1,0.189375,0.017500,2014.300000,320.078750
3,0.211765,0.016471,2236.085882,316.093529
5,0.192941,0.014118,2076.454118,330.925294


### 2.1.1. SELECTED PARAMS

In [ ]:
param = {
    'n_estimatores': [1000, 1300, 1700],
    'learning_rate': [0.08, 0.11, 0.14], 
    'max_depth': [9, 12],
    'subsample': [0.5],  
    'colsample_bytee': [0.5],
    'colsample_bynode': [0.5],
    'min_child_weight': [5]
}

## 2.2. SECOND ROUND

In [30]:
df01 = data.copy()

In [31]:
# id's 10000 --> selected params of first run
df01 = df01[(df01['id'] > 10000) & (df01['id'] < 20000)]

df01 = create_mape_rmse_columns(df01)

list_columns = ['n_estimatores', 'learning_rate', 'max_depth']

display_errors_by_parameters(df01, list_columns)

,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
n_estimatores,,,,
1000,0.138333,0.018333,1475.536667,292.238333
1300,0.138333,0.018333,1475.536667,292.238333
1700,0.138333,0.018333,1475.536667,292.238333


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
learning_rate,,,,
0.08,0.150,0.015,1612.275,302.300
0.11,0.135,0.020,1433.575,286.405
0.14,0.130,0.020,1380.760,288.010


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
max_depth,,,,
9,0.153333,0.016667,1599.103333,282.186667
12,0.123333,0.020000,1351.970000,302.290000


### 2.2.1. SELECTED PARAMS

In [ ]:
param = {
    'n_estimatores': [1300, 1500],
    'learning_rate': [0.11, 0.14, 0.17], 
    'max_depth': [12, 15],
    'subsample': [0.5],  
    'colsample_bytee': [0.5],
    'colsample_bynode': [0.5],
    'min_child_weight': [5]
}

## 2.3. LAST ROUND

In [22]:
df02 = data.copy()
df03 = data.copy()

In [23]:
# id's 20000 --> selected params of second run
df02 = df02[(df02['id'] > 20000) & (df02['id'] < 30000)]
# id' 30000 -> selected params of second run--> but using new param to test--> num_parallel_tree = 2
df03 = df03[(df03['id'] > 30000) & (df03['id'] < 40000)]

df02 = create_mape_rmse_columns(df02)

df03 = create_mape_rmse_columns(df02)

list_columns = ['n_estimatores', 'learning_rate', 'max_depth']

Selected params of second run

In [32]:
display_errors_by_parameters(df02, list_columns)

,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
n_estimatores,,,,
1300,0.12,0.02,1282.271667,300.011667
1500,0.12,0.02,1282.271667,300.011667


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
learning_rate,,,,
0.11,0.12,0.02,1280.500,303.240
0.14,0.12,0.02,1279.760,312.385
0.17,0.12,0.02,1286.555,284.410


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
max_depth,,,,
12,0.12,0.02,1298.566667,297.333333
15,0.12,0.02,1265.976667,302.690000


Selected params of second run--> but using new param to test--> num_parallel_tree = 2

In [33]:
display_errors_by_parameters(df03, list_columns)

,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
n_estimatores,,,,
1300,0.12,0.02,1282.271667,300.011667
1500,0.12,0.02,1282.271667,300.011667


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
learning_rate,,,,
0.11,0.12,0.02,1280.500,303.240
0.14,0.12,0.02,1279.760,312.385
0.17,0.12,0.02,1286.555,284.410


,mean_MAPE,std_MAPE,mean_RMSE,std_RMSE
max_depth,,,,
12,0.12,0.02,1298.566667,297.333333
15,0.12,0.02,1265.976667,302.690000


### 2.2.1. SELECTED PARAMS

In [ ]:
param = {
    'n_estimatores': 1300,
    'learning_rate': 0.11, 
    'max_depth': 12,
    'subsample': 0.5,  
    'colsample_bytee': 0.5,
    'colsample_bynode': 0.5,
    'min_child_weight': 5
}